# M3 Model Discovery and Extraction

This notebook demonstrates how to access HEC-HMS projects from the Harris County Flood Control District (HCFCD) M3 Models.

**M3 Models** are FEMA-effective H&H (Hydrology & Hydraulics) models for major bayous and watersheds in the Houston, Texas region. Each model contains:
- **HEC-HMS projects**: Hydrologic models for runoff generation
- **HEC-RAS projects**: Hydraulic models for river routing

## Prerequisites

**Internet Required**: This notebook downloads data from the HCFCD M3 model library.
- Model downloads: 25-100 MB per model
- Estimated time: 2-5 minutes for first extraction (depends on connection speed)

**Estimated Total Runtime**: 5-10 minutes

## What You'll Learn

1. Listing available M3 models and HMS projects
2. Querying project metadata
3. Finding projects by channel name
4. Extracting HMS projects for use
5. Integration with hms-commander workflows

In [1]:
# pip install hms-commander

**For Development**: If working on hms-commander source code, use the `hmscmdr_local` conda environment (editable install) instead of pip install.

**Note**: This notebook **requires M3 data download**. The first extraction will download model archives from the HCFCD model library (~25-100 MB per model).

## Setup

In [2]:
from pathlib import Path

from hms_commander import HmsM3Model, init_hms_project, __version__
import pandas as pd

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print(f"hms-commander v{__version__}")

hms-commander v0.1.0


## 1. List Available M3 Models

M3 Models are identified by single letters (A-W, excluding V). Each model covers a specific watershed or bayou in Harris County.

In [3]:
# List all M3 models that contain HMS projects
models = HmsM3Model.list_models()
print(f"Total M3 Models with HMS projects: {len(models)}\n")
models[['model_id', 'name', 'hms_project_count', 'effective_date']]

2026-01-08 13:27:48 - hms_commander.HmsM3Model - INFO - Listed 22 M3 Models with HMS projects


Total M3 Models with HMS projects: 22



,model_id,name,hms_project_count,effective_date
0,A,Clear Creek,1,2022-05-05
1,B,Armand Bayou,1,2022-05-05
2,C,Sims Bayou,1,2022-05-05
3,D,Brays Bayou,1,2022-05-05
4,E,White Oak Bayou,1,2023-01-30
5,F,San Jacinto/Galveston Bay,2,2022-05-05
6,G,San Jacinto River,16,2022-05-05
7,H,Hunting Bayou,1,2022-05-05
8,I,Vince Bayou,1,2022-05-05
9,J,Spring Creek,1,2022-05-05


## 2. List HMS Projects

Each M3 model contains one or more HMS projects. The catalog includes metadata about each project.

In [4]:
# List all HMS projects across all models
all_projects = HmsM3Model.list_projects()
print(f"Total HMS projects: {len(all_projects)}\n")
all_projects[['model_id', 'model_name', 'unit_id', 'hms_version', 'loss_method', 'transform_method']].head(15)

2026-01-08 13:27:48 - hms_commander.HmsM3Model - INFO - Listed 42 HMS projects


Total HMS projects: 42



,model_id,model_name,unit_id,hms_version,loss_method,transform_method
0,A,Clear Creek,A100-00-00,3.3,Green and Ampt,Clark
1,B,Armand Bayou,B100-00-00,3.3,Green and Ampt,Clark
2,C,Sims Bayou,C100-00-00,3.3,Initial+Constant,Clark
3,D,Brays Bayou,D100-00-00,3.3,Initial+Constant,Clark
4,E,White Oak Bayou,E100-00-00,3.3,Initial+Constant,Clark
5,F,San Jacinto/Galveston Bay,F216-00-00,3.3,Initial+Constant,Clark
6,F,San Jacinto/Galveston Bay,F220-00-00,3.3,Initial+Constant,Clark
7,G,San Jacinto River,G100-00-00,3.3,Initial+Constant,Clark
8,G,San Jacinto River,G103-01-00,3.3,Initial+Constant,Clark
9,G,San Jacinto River,G103-07-00,3.3,Initial+Constant,Clark


In [5]:
# List projects for a specific model (Model G has the most)
san_jacinto_projects = HmsM3Model.list_projects(model_id='G')
print(f"San Jacinto River (Model G) HMS projects: {len(san_jacinto_projects)}\n")
san_jacinto_projects[['unit_id', 'hms_file', 'hms_version']]

2026-01-08 13:27:48 - hms_commander.HmsM3Model - INFO - Listed 16 HMS projects


San Jacinto River (Model G) HMS projects: 16



,unit_id,hms_file,hms_version
7,G100-00-00,G1000000.hms,3.3
8,G103-01-00,G1030100.hms,3.3
9,G103-07-00,G103_07_00.hms,3.3
10,G103-07-00-alt,G1030700.hms,3.3
11,G103-33-00,G1033300.hms,3.3
12,G103-43-00,G1034300.hms,3.3
13,G103-44-00,G1034400.hms,3.3
14,G103-48-00,G1034800.hms,3.3
15,G103-80-31,G1038031.hms,3.3
16,G103-80-32,G1038032.hms,3.3


## 3. Get Model and Project Information

Get detailed metadata about specific models and projects.

In [6]:
# Get detailed info about Brays Bayou (Model D)
model_info = HmsM3Model.get_model_info('D')

print("=== Model D: Brays Bayou ===")
print(f"Name: {model_info['name']}")
print(f"Effective Date: {model_info['effective_date']}")
print(f"Download Size: {model_info['size_gb']} GB")
print(f"Primary Channels: {model_info['primary_channels']}")
print(f"HMS Projects: {model_info['hms_projects']}")

2026-01-08 13:27:48 - hms_commander.HmsM3Model - INFO - Retrieved info for model 'D': Brays Bayou


=== Model D: Brays Bayou ===
Name: Brays Bayou
Effective Date: 2022-05-05
Download Size: 0.03 GB
Primary Channels: ['BRAYS BAYOU']
HMS Projects: ['D100-00-00']


In [7]:
# Get detailed info about a specific HMS project
project_info = HmsM3Model.get_project_info('D', 'D100-00-00')

print("=== Project D100-00-00 ===")
for key, value in project_info.items():
    print(f"{key}: {value}")

2026-01-08 13:27:48 - hms_commander.HmsM3Model - INFO - Retrieved info for project 'D100-00-00'


=== Project D100-00-00 ===
model_id: D
model_name: Brays Bayou
unit_id: D100-00-00
hms_file: D100_00_00.hms
project_name: D100_00_00
description: HCFCD Unit # D100-00-00
hms_version: 3.3
unit_system: Metric
loss_method: Initial+Constant
transform_method: Clark
baseflow_method: Recession
routing_method: Lag
design_storms: 0.2%,1%,2%,10%
rainfall_region: Region 3
dss_file: D100_00_00.dss
relative_path: D/HEC-HMS/D_D100-00-00/D_D100-00-00


## 4. Find Projects by Channel Name

You can find M3 models and HMS projects by searching for channel names.

In [8]:
# Find HMS project by channel name
channels_to_find = [
    'BRAYS BAYOU',
    'BUFFALO BAYOU',
    'WHITE OAK BAYOU',
    'GREENS BAYOU',
    'CLEAR CREEK'
]

print("Channel Name Lookups:")
print("-" * 60)
for channel in channels_to_find:
    result = HmsM3Model.get_project_by_channel(channel)
    if result:
        model_id, unit_id = result
        model_name = HmsM3Model.MODELS[model_id]['name']
        print(f"{channel:20} -> Model {model_id} ({model_name}), Unit {unit_id}")
    else:
        print(f"{channel:20} -> Not found")

2026-01-08 13:27:48 - hms_commander.HmsM3Model - INFO - Channel 'BRAYS BAYOU' -> Model D, Unit D100-00-00


2026-01-08 13:27:48 - hms_commander.HmsM3Model - INFO - Channel 'BUFFALO BAYOU' -> Model W, Unit W100-00-00


2026-01-08 13:27:48 - hms_commander.HmsM3Model - INFO - Channel 'WHITE OAK BAYOU' -> Model E, Unit E100-00-00


2026-01-08 13:27:48 - hms_commander.HmsM3Model - INFO - Channel 'GREENS BAYOU' -> Model P, Unit P100-00-00


2026-01-08 13:27:48 - hms_commander.HmsM3Model - INFO - Channel 'CLEAR CREEK' -> Model A, Unit A100-00-00


Channel Name Lookups:
------------------------------------------------------------
BRAYS BAYOU          -> Model D (Brays Bayou), Unit D100-00-00
BUFFALO BAYOU        -> Model W (Buffalo Bayou), Unit W100-00-00
WHITE OAK BAYOU      -> Model E (White Oak Bayou), Unit E100-00-00
GREENS BAYOU         -> Model P (Greens Bayou), Unit P100-00-00
CLEAR CREEK          -> Model A (Clear Creek), Unit A100-00-00


## 5. Catalog Statistics

Get summary statistics about the HMS project catalog.

In [9]:
# Get catalog statistics
stats = HmsM3Model.get_statistics()

print("=== M3 HMS Catalog Statistics ===")
print(f"Total HMS Projects: {stats['total_projects']}")
print(f"Total M3 Models: {stats['total_models']}")
print(f"\nModels with Most HMS Projects:")
for model_id, count in stats['models_with_most_projects'].items():
    print(f"  Model {model_id}: {count} projects")
print(f"\nHMS Versions:")
for version, count in stats['hms_versions'].items():
    print(f"  {version}: {count} projects")
print(f"\nLoss Methods:")
for method, count in stats['loss_methods'].items():
    print(f"  {method}: {count} projects")

=== M3 HMS Catalog Statistics ===
Total HMS Projects: 42
Total M3 Models: 22

Models with Most HMS Projects:
  Model G: 16 projects
  Model F: 2 projects
  Model O: 2 projects

HMS Versions:
  3.3: 41 projects
  3.4: 1 projects

Loss Methods:
  Initial+Constant: 40 projects
  Green and Ampt: 2 projects


## 6. Extract HMS Project

To use an HMS project, extract it from the M3 archive. This downloads the model zip and extracts the HMS portion.

**Note**: First extraction downloads ~25-100 MB per model.

In [10]:
# Extract Brays Bayou HMS project
# This downloads ~32 MB and extracts the HMS files

path = HmsM3Model.extract_project(
    'D', 
    'D100-00-00',
    output_path=Path.cwd() / 'hms_example_projects' / 'm3_brays_bayou'
)
print(f"Extracted to: {path}")

# List extracted files
hms_files = list(Path(path).glob('*.hms'))
print(f"\nHMS project file: {hms_files[0].name if hms_files else 'Not found'}")
print(f"\nAll files ({len(list(Path(path).glob('*')))} total):")
for f in sorted(Path(path).glob('*'))[:10]:
    print(f"  {f.name}")
if len(list(Path(path).glob('*'))) > 10:
    print("  ...")

2026-01-08 13:27:48 - hms_commander.HmsM3Model - INFO - Retrieved info for project 'D100-00-00'


2026-01-08 13:27:48 - hms_commander.HmsM3Model - INFO - ----- HmsM3Model Extracting Project -----


2026-01-08 13:27:48 - hms_commander.HmsM3Model - INFO - Model: D - Brays Bayou


2026-01-08 13:27:48 - hms_commander.HmsM3Model - INFO - Unit: D100-00-00


2026-01-08 13:27:48 - hms_commander.HmsM3Model - INFO - Project already exists at C:\GH\hms-commander\examples\hms_example_projects\m3_brays_bayou\D\D100-00-00


2026-01-08 13:27:48 - hms_commander.HmsM3Model - INFO - Use overwrite=True to re-download


Extracted to: C:\GH\hms-commander\examples\hms_example_projects\m3_brays_bayou\D\D100-00-00

HMS project file: D100_00_00.hms

All files (32 total):
  .2PCT.log
  0.2__24HR.met
  0.2PCT.log
  10__24HR.met
  10PCT.log
  1__24HR.met
  1PCT.log
  2007_Effective_D_D100-00-00_ReadMe.doc
  2__24HR.met
  2PCT.log
  ...


In [11]:
# Check if a project is already extracted
is_extracted = HmsM3Model.is_project_extracted('D', 'D100-00-00')
print(f"D100-00-00 extracted: {is_extracted}")

D100-00-00 extracted: True


## 7. Integration with hms-commander Workflow

Once extracted, use the HMS project with standard hms-commander operations.

**Important**: M3 HMS projects use HMS 3.x format (version 3.3 or 3.4). When generating Jython scripts, you must use `python2_compatible=True`.

In [12]:
# Initialize the extracted project
if path and Path(path).exists():
    hms = init_hms_project(path)
    
    print(f"Project: {hms.project_name}")
    print(f"HMS Version: {hms.hms_version}")
    print(f"\nBasin models: {len(hms.basin_df)}")
    print(f"Met models: {len(hms.met_df)}")
    print(f"Control specs: {len(hms.control_df)}")
    print(f"Runs: {len(hms.run_df)}")
    
    print(f"\nAvailable runs:")
    for run_name in hms.list_run_names():
        print(f"  - {run_name}")

2026-01-08 13:27:48 - hms_commander.HmsPrj - INFO - HMS project initialized: D100_00_00


2026-01-08 13:27:48 - hms_commander.HmsPrj - INFO -   Version: 3.3


2026-01-08 13:27:48 - hms_commander.HmsPrj - INFO -   Basin models: 4


2026-01-08 13:27:48 - hms_commander.HmsPrj - INFO -   Met models: 4


2026-01-08 13:27:48 - hms_commander.HmsPrj - INFO -   Control specs: 1


2026-01-08 13:27:48 - hms_commander.HmsPrj - INFO -   Simulation runs: 4


2026-01-08 13:27:48 - hms_commander.HmsPrj - INFO -   Gages: 0


2026-01-08 13:27:48 - hms_commander.HmsPrj - INFO -   Paired data tables: 60


Project: D100_00_00
HMS Version: 3.3

Basin models: 4
Met models: 4
Control specs: 1
Runs: 4

Available runs:
  - .2PCT
  - 10PCT
  - 1PCT
  - 2PCT


### Example Execution Workflow (HMS 3.x)

```python
from hms_commander import HmsM3Model, init_hms_project, HmsJython, HmsResults

# 1. Extract project (downloads ~32 MB)
path = HmsM3Model.extract_project('D', 'D100-00-00')

# 2. Initialize project
hms = init_hms_project(path)
print(f"Project: {hms.project_name}, Version: {hms.hms_version}")
print(f"Available runs: {hms.list_run_names()}")

# 3. Generate execution script (MUST use python2_compatible for HMS 3.x)
script = HmsJython.generate_compute_script(
    project_path=path,
    run_name="1PCT",
    python2_compatible=True  # Required for M3 HMS projects!
)

# 4. Execute (requires HMS 3.x installation)
# HmsCmdr.compute_run("1PCT", python2_compatible=True)

# 5. Extract results
dss_file = path / "D100_00_00.dss"
peaks = HmsResults.get_peak_flows(dss_file)
print(peaks)
```

## 8. HMS-RAS Integration

M3 Models contain both HMS (hydrology) and RAS (hydraulics) projects. Use `ras-commander` for RAS operations.

**Typical Workflow**:
1. Run HMS to generate flow hydrographs
2. Import HMS DSS results as RAS upstream boundary conditions
3. Run RAS for water surface profiles

In [13]:
# Cross-reference HMS and RAS for same watershed
print("=== Brays Bayou (Model D) - HMS and RAS Projects ===")
print()

# HMS side
hms_info = HmsM3Model.get_model_info('D')
print(f"HMS Projects: {hms_info['hms_projects']}")
print(f"  - 1 HMS project covers entire watershed hydrology")
print()

# RAS side (informational - requires ras-commander)
print("RAS Reaches (via ras-commander M3Model):")
print("  D100-00-00, D109-00-00, D111-00-00, D112-00-00,")
print("  D118-00-00, D120-00-00, D122-00-00, D124-00-00,")
print("  D126-00-00, D129-00-00, D132-00-00, D133-00-00,")
print("  D139-00-00, D140-00-00, D142-00-00, D144-00-00")
print(f"  - 16 RAS reaches for detailed hydraulics")

2026-01-08 13:27:48 - hms_commander.HmsM3Model - INFO - Retrieved info for model 'D': Brays Bayou


=== Brays Bayou (Model D) - HMS and RAS Projects ===

HMS Projects: ['D100-00-00']
  - 1 HMS project covers entire watershed hydrology

RAS Reaches (via ras-commander M3Model):
  D100-00-00, D109-00-00, D111-00-00, D112-00-00,
  D118-00-00, D120-00-00, D122-00-00, D124-00-00,
  D126-00-00, D129-00-00, D132-00-00, D133-00-00,
  D139-00-00, D140-00-00, D142-00-00, D144-00-00
  - 16 RAS reaches for detailed hydraulics


## Summary

`HmsM3Model` provides access to 42 HMS projects across 22 HCFCD M3 Models:

| Feature | Description |
|---------|-------------|
| Models | 22 M3 models (A-W) with HMS content |
| Projects | 42 HMS projects total |
| HMS Version | Mostly 3.3 (requires `python2_compatible=True`) |
| Design Storms | 0.2%, 1%, 2%, 10% AEP typically |
| Unit System | Mostly Metric |

**Key Methods**:

| Method | Purpose |
|--------|--------|
| `list_models()` | List M3 models with HMS projects |
| `list_projects()` | List all HMS projects |
| `get_model_info()` | Get model metadata |
| `get_project_info()` | Get project metadata |
| `get_project_by_channel()` | Find by channel name |
| `extract_project()` | Download and extract HMS project |

## Next Steps

- **09_m3_conversion.ipynb**: Convert HMS 3.x to 4.x with QAQC
- **07_execution_jython.ipynb**: Advanced execution patterns
- **HCFCD Resources**: https://www.m3models.org/